# Mp3 Tagger

 Parses mp3 file names and adds ID3 tags based on file name and user input.

## Strategy
* Open group of files
* Parse names into parts
* Create dictionary of ID3 tags for each file
* Determine easy metadata by repeating occurrances in name-parts
    * Artist name: most frequent
    * Album name: second most
    * Track number: name-part is just a number?, count total in album for second part
    * Album art?
    * Genre?


In [1]:
import mutagen
import os
import numpy
import librosa
import torch


ModuleNotFoundError: No module named 'librosa'

In [ ]:
# librosa to tensor

# 1. Generate Mel Spectrogram with Librosa
y, sr = librosa.load("song.mp3")
melspec_np = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128)

# 2. Convert to PyTorch Tensor
# Use .float() because neural nets usually expect float32
melspec_tensor = torch.from_numpy(melspec_np).float()

# 3. Add Batch and Channel dimensions (required for most Conv2D layers)
# Resulting shape: [1, 1, 128, time_steps]
melspec_tensor = melspec_tensor.unsqueeze(0).unsqueeze(0)